In [10]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [11]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from gsm import GaussianSoftmaxModel
from rsm import RecurrentStickbreakingModel
from evaluation import validate, print_flat_topic_sample
from configure import get_config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load data & set config

In [12]:
config = get_config(nb_name)

In [13]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [14]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [15]:
def debug_shape(variables, model):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, model, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    return _variables

# run

## initialize log

In [16]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','VALID:','','','','','TEST:',''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL','REG','LOSS','PPL','NLL','KL','REG','LOSS','PPL']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize model

In [17]:
if 'sess' in globals(): sess.close()
if config.model == 'gsm':
    Model = GaussianSoftmaxModel
elif config.model == 'rsm':
    Model = RecurrentStickbreakingModel
model = Model(config)    
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

## train & validate model

In [18]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

        if global_step_log % config.log_period == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topics_freq_tokens = [[idx_to_word[idx] for idx in topic_freq_idxs] for topic_freq_idxs in topics_freq_idxs]
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = Model(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

display(log_df)
print_flat_topic_sample(sess, model, topics_freq_tokens=topics_freq_tokens)

TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     38   28   99  599.03  882  595.48  3.46  0.10  581.37  954  577.67   
10000    37   57   24  598.63  866  594.46  4.11  0.07  572.34  895  567.97   
15000    37   85  124  596.87  856  592.44  4.38  0.05  573.97  890  569.38   
20000    36  114   49  596.76  850  592.19  4.53  0.04  571.47  883  566.88   
25000    35  142  149  596.19  846  591.52  4.63  0.04  573.31  885  568.59   
30000    37  171   74  595.86  843  591.14  4.69  0.03  573.85  880  569.12   
35000    37  199  174  595.93  840  591.15  4.75  0.03  571.92  879  567.23   
40000    36  228   99  595.45  839  590.64  4.78  0.03  580.48  916  575.82   
45000    36  257   24  595.52  837  590.68  4.81  0.03  571.09  871  566.32   
50000    36  285  124  595.19  836  590.34  4.83  0.03  575.89  886  571.22   
55000    36  314   49  595.24  835  590.37  4.84  0.03  571.43  873  566.78   
60000    36  342  149  595.09  834  590.21  4.86  0.03  571.54  871  566.85   
65000    36  371   74  595.01  833  590.11  4.87  0.02  575.19  887  570.48   
70000    34  399  174  595.09  832  590.17  4.89  0.02  572.54  876  567.83   
75000    36  428   99  594.90  832  589.97  4.90  0.02  583.63  936  578.88   
80000    37  457   24  594.97  831  590.03  4.91  0.02  572.68  879  567.89   
85000    37  485  124  594.82  831  589.87  4.92  0.02  577.95  900  573.14   
90000    37  514   49  594.89  831  589.93  4.94  0.02  573.01  882  568.20   
95000    37  542  149  594.83  830  589.85  4.95  0.02  572.31  873  567.48   
100000   36  571   74  594.80  830  589.81  4.96  0.02  574.78  884  569.97   
105000   36  599  174  594.87  830  589.88  4.97  0.02  572.90  880  568.13   
110000   36  628   99  594.75  830  589.75  4.98  0.02  584.08  938  579.19   
115000   36  657   24  594.82  829  589.81  4.99  0.02  573.33  878  568.42   
120000   36  685  124  594.73  829  589.70  5.00  0.02  577.87  899  573.03   
125000   36  714   49  594.79  829  589.74  5.01  0.02  572.71  879  567.85   
130000   36  742  149  594.75  829  589.69  5.01  0.02  572.16  873  567.30   
135000   36  771   74  594.74  829  589.66  5.02  0.02  575.58  887  570.78   
140000   36  799  174  594.81  829  589.71  5.03  0.02  574.95  890  570.01   
145000   37  828   99  594.73  829  589.62  5.04  0.02  583.08  929  578.13   
150000   36  857   24  594.79  829  589.67  5.05  0.02  572.75  879  567.82   
155000   36  885  124  594.73  829  589.59  5.06  0.02  580.12  911  575.20   
160000   36  914   49  594.79  829  589.63  5.07  0.02  573.22  882  568.26   
165000   37  942  149  594.77  829  589.60  5.07  0.02  573.43  881  568.49   
170000   38  971   74  594.77  829  589.59  5.08  0.02  574.32  886  569.41   
175000   38  999  174  594.83  829  589.63  5.09  0.02  573.53  885  568.52   

                     TEST:       
          KL   REG    LOSS  PPL  
5000    3.65  0.05  581.60  956  
10000   4.34  0.03  572.34  894  
15000   4.57  0.02  574.01  887  
20000   4.58  0.02  571.52  882  
25000   4.70  0.02  571.52  882  
30000   4.72  0.02  573.74  881  
35000   4.67  0.01  572.09  879  
40000   4.64  0.01  572.09  879  
45000   4.75  0.02  571.02  872  
50000   4.66  0.01  571.02  872  
55000   4.63  0.01  571.02  872  
60000   4.68  0.01  571.02  872  
65000   4.69  0.02  571.02  872  
70000   4.70  0.01  571.02  872  
75000   4.74  0.01  571.02  872  
80000   4.78  0.01  571.02  872  
85000   4.79  0.01  571.02  872  
90000   4.79  0.01  571.02  872  
95000   4.82  0.01  571.02  872  
100000  4.79  0.01  571.02  872  
105000  4.76  0.01  571.02  872  
110000  4.88  0.01  571.02  872  
115000  4.90  0.01  571.02  872  
120000  4.83  0.01  571.02  872  
125000  4.85  0.01  571.02  872  
130000  4.85  0.01  571.02  872  
135000  4.79  0.01  571.02  872  
140000  4.93  0.01  571.02  872  
145000  4.94  0.01  571.02  872  
150000  4.92  0.01  571.02  872  
155000  4.9

0 use get file write one work drive also program system
1 write go get article one think say know like see
2 god say one christian people believe write jesus know think
3 game team play player year win hockey season league pt
4 key use chip encryption government law clipper system privacy information
5 use car health one disease food may medical cause system
6 write people one article think say system make law man
7 israel people turkish armenian israeli armenians jews say war government
8 space launch nasa satellite orbit mission system earth moon lunar
9 gun government people president state weapon law use make stephanopoulos


TRAIN:                           VALID:               \
       Time   Ep   Ct    LOSS  PPL     NLL    KL   REG    LOSS  PPL     NLL   
5000     38   28   99  599.03  882  595.48  3.46  0.10  581.37  954  577.67   
10000    37   57   24  598.63  866  594.46  4.11  0.07  572.34  895  567.97   
15000    37   85  124  596.87  856  592.44  4.38  0.05  573.97  890  569.38   
20000    36  114   49  596.76  850  592.19  4.53  0.04  571.47  883  566.88   
25000    35  142  149  596.19  846  591.52  4.63  0.04  573.31  885  568.59   
30000    37  171   74  595.86  843  591.14  4.69  0.03  573.85  880  569.12   
35000    37  199  174  595.93  840  591.15  4.75  0.03  571.92  879  567.23   
40000    36  228   99  595.45  839  590.64  4.78  0.03  580.48  916  575.82   
45000    36  257   24  595.52  837  590.68  4.81  0.03  571.09  871  566.32   
50000    36  285  124  595.19  836  590.34  4.83  0.03  575.89  886  571.22   
55000    36  314   49  595.24  835  590.37  4.84  0.03  571.43  873  566.78   
60000    36  342  149  595.09  834  590.21  4.86  0.03  571.54  871  566.85   
65000    36  371   74  595.01  833  590.11  4.87  0.02  575.19  887  570.48   
70000    34  399  174  595.09  832  590.17  4.89  0.02  572.54  876  567.83   
75000    36  428   99  594.90  832  589.97  4.90  0.02  583.63  936  578.88   
80000    37  457   24  594.97  831  590.03  4.91  0.02  572.68  879  567.89   
85000    37  485  124  594.82  831  589.87  4.92  0.02  577.95  900  573.14   
90000    37  514   49  594.89  831  589.93  4.94  0.02  573.01  882  568.20   
95000    37  542  149  594.83  830  589.85  4.95  0.02  572.31  873  567.48   
100000   36  571   74  594.80  830  589.81  4.96  0.02  574.78  884  569.97   
105000   36  599  174  594.87  830  589.88  4.97  0.02  572.90  880  568.13   
110000   36  628   99  594.75  830  589.75  4.98  0.02  584.08  938  579.19   
115000   36  657   24  594.82  829  589.81  4.99  0.02  573.33  878  568.42   
120000   36  685  124  594.73  829  589.70  5.00  0.02  577.87  899  573.03   
125000   36  714   49  594.79  829  589.74  5.01  0.02  572.71  879  567.85   
130000   36  742  149  594.75  829  589.69  5.01  0.02  572.16  873  567.30   
135000   36  771   74  594.74  829  589.66  5.02  0.02  575.58  887  570.78   
140000   36  799  174  594.81  829  589.71  5.03  0.02  574.95  890  570.01   
145000   37  828   99  594.73  829  589.62  5.04  0.02  583.08  929  578.13   
150000   36  857   24  594.79  829  589.67  5.05  0.02  572.75  879  567.82   
155000   36  885  124  594.73  829  589.59  5.06  0.02  580.12  911  575.20   
160000   36  914   49  594.79  829  589.63  5.07  0.02  573.22  882  568.26   
165000   37  942  149  594.77  829  589.60  5.07  0.02  573.43  881  568.49   
170000   38  971   74  594.77  829  589.59  5.08  0.02  574.32  886  569.41   
175000   38  999  174  594.83  829  589.63  5.09  0.02  573.53  885  568.52   

                     TEST:       
          KL   REG    LOSS  PPL  
5000    3.65  0.05  581.60  956  
10000   4.34  0.03  572.34  894  
15000   4.57  0.02  574.01  887  
20000   4.58  0.02  571.52  882  
25000   4.70  0.02  571.52  882  
30000   4.72  0.02  573.74  881  
35000   4.67  0.01  572.09  879  
40000   4.64  0.01  572.09  879  
45000   4.75  0.02  571.02  872  
50000   4.66  0.01  571.02  872  
55000   4.63  0.01  571.02  872  
60000   4.68  0.01  571.02  872  
65000   4.69  0.02  571.02  872  
70000   4.70  0.01  571.02  872  
75000   4.74  0.01  571.02  872  
80000   4.78  0.01  571.02  872  
85000   4.79  0.01  571.02  872  
90000   4.79  0.01  571.02  872  
95000   4.82  0.01  571.02  872  
100000  4.79  0.01  571.02  872  
105000  4.76  0.01  571.02  872  
110000  4.88  0.01  571.02  872  
115000  4.90  0.01  571.02  872  
120000  4.83  0.01  571.02  872  
125000  4.85  0.01  571.02  872  
130000  4.85  0.01  571.02  872  
135000  4.79  0.01  571.02  872  
140000  4.93  0.01  571.02  872  
145000  4.94  0.01  571.02  872  
150000  4.92  0.01  571.02  872  
155000  4.9

0 use get file write one work drive also program system
1 write go get article one think say know like see
2 god say one christian people believe write jesus know think
3 game team play player year win hockey season league pt
4 key use chip encryption government law clipper system privacy information
5 use car health one disease food may medical cause system
6 write people one article think say system make law man
7 israel people turkish armenian israeli armenians jews say war government
8 space launch nasa satellite orbit mission system earth moon lunar
9 gun government people president state weapon law use make stephanopoulos
